In [24]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch
import re, os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, NLTKTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from langchain.embeddings import GPT4AllEmbeddings
import os
import shutil

In [25]:
# searchType = "Main"
# search_query = "What are your refund policies?"

searchType = "Seller"
search_query = "Can I make product bundles on Daraz?"

search_query = re.sub(r'\bDaraz\b\s*', '', search_query, flags=re.IGNORECASE)
chunkSize = 1500

In [26]:
if os.path.exists("/Users/moiz/Moiz/Github/ITA-Project/outputCleaned.txt"):
    os.remove("/Users/moiz/Moiz/Github/ITA-Project/outputCleaned.txt")

In [27]:
def is_paragraph_break(line):
    return line.strip() == ""

def is_unwanted_line(line):
    # Check if a line ends with a colon
    return line.strip().endswith(":")

def process_files(folder_path, output_file):
    files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    all_text = []
    
    for file in files:
        current_paragraph = []
        with open(os.path.join(folder_path, file), 'r', encoding='utf-8') as f:
            for line in f:
                if is_unwanted_line(line):
                    continue  # Skip lines ending with a colon
                if is_paragraph_break(line):
                    if current_paragraph:
                        all_text.append(" ".join(current_paragraph))
                        current_paragraph = []
                else:
                    # Remove leading/trailing whitespace and add the line to the current paragraph
                    current_paragraph.append(line.strip())
            # Don't forget to add the last paragraph if the file didn't end with a blank line
            if current_paragraph:
                all_text.append(" ".join(current_paragraph))
    
    # Remove lines with less than 100 characters
    all_text = [line for line in all_text if len(line) >= 100]
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(all_text))

if searchType.lower() == "main":
    folder_path = '/Users/moiz/Moiz/Github/ITA-Project/DarazDataMain'  # Change this to the path of your folder
    output_file = 'DarazDataMain.txt'  
    process_files(folder_path, output_file)
    file_name = "DarazDataMain.txt"
elif searchType.lower() == "seller":
    folder_path = '/Users/moiz/Moiz/Github/ITA-Project/DarazDataSeller'  # Change this to the path of your folder
    output_file = 'DarazDataSeller.txt'  
    process_files(folder_path, output_file)
    file_name = "DarazDataSeller.txt"

with open(file_name, 'r', encoding="utf8") as f:
    products = f.readlines()

In [28]:
model = SentenceTransformer('all-MiniLM-L6-v2')
product_embeddings = model.encode(products, convert_to_tensor=True)

def search_products(query, k):
    query_embedding = model.encode(query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, product_embeddings)[0]
    top_results = torch.topk(cosine_scores, k=k)

    print("Query:", query)
    for score, idx in zip(top_results[0], top_results[1]):
        print("\nScore:", score.item())
        print("Product Details:", products[idx])

search_products(search_query, 2)

Query: Can I make product bundles on ?

Score: 0.6566495895385742
Product Details: Why should I create bundles? Sellers can create bundles to differentiate their deals from competitors, especially in high-priced competitive categories It's a great opportunity to sell low-cost items as part of a bundle Bundles allow sellers to clear their inventory of whatever products they choose Additionally, bundles are a foolproof way to attract customers and increase conversion rate, sales and enhance overall performance.


Score: 0.6418728828430176
Product Details: What are the Benefits of creating Product Bundles? Sellers can create bundles to differentiate their deals from competitors, especially in high-priced competitive categories It’s a great opportunity to sell low cost items as part of a bundle Bundles allow Sellers to clear their inventory of whatever products they choose Additionally, bundles are a full proof way to attract customers and increase conversion rate, sales and enhance overal

In [29]:
query = search_query
loader = TextLoader(file_name)
docs = loader.load()

In [30]:
import os
import re
import shutil

def manage_chroma_folders(directory):
    global chromaCounter
    chromaCounter = 0  # Initialize the counter to zero

    # Prepare to capture the highest number found in folder names starting with "chromaDB"
    highest_number = 0

    # Compile a regex pattern to match 'chromaDB' followed immediately by a number
    pattern = re.compile(r'^chromaDB(\d+)$')

    # List all items in the directory
    items = os.listdir(directory)

    # First pass: Identify the highest number suffix for "chromaDB" folders
    for item in items:
        if os.path.isdir(os.path.join(directory, item)):
            match = pattern.match(item)
            if match:
                # Extract the numeric part and update the highest_number if this one is greater
                number = int(match.group(1))
                if number > highest_number:
                    highest_number = number

    # Update the chromaCounter to the next available number
    chromaCounter = highest_number + 1

    # Second pass: Delete all folders starting with "chroma"
    for item in items:
        if os.path.isdir(os.path.join(directory, item)) and item.startswith("chroma"):
            full_path = os.path.join(directory, item)
            shutil.rmtree(full_path)
            # print(f"Deleted folder: {full_path}")

# Usage example:
directory = "/Users/moiz/Moiz/Github/ITA-Project/"
manage_chroma_folders(directory)
# print(f"Next chromaCounter value: {chromaCounter}")

with open('chromaCounter.txt', 'w') as file:
    file.write(f"Current Chroma Counter = {chromaCounter}")

In [31]:
chunker01 = "Fixed-size (in characters) Overlapping Sliding Window"

text_splitter = CharacterTextSplitter(separator="\n", chunk_size=chunkSize, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

persist_directory = 'chromaDB' + str(chromaCounter) + '/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings(),
    persist_directory=persist_directory
)
vectordb.persist()
docs01 = vectordb.similarity_search_with_score(query, k=2)

Created a chunk of size 2262, which is longer than the specified 1500
Created a chunk of size 7956, which is longer than the specified 1500
Created a chunk of size 2841, which is longer than the specified 1500
Created a chunk of size 2920, which is longer than the specified 1500
Created a chunk of size 2399, which is longer than the specified 1500
Created a chunk of size 1538, which is longer than the specified 1500
Created a chunk of size 1731, which is longer than the specified 1500
Created a chunk of size 2714, which is longer than the specified 1500
Created a chunk of size 1561, which is longer than the specified 1500
Created a chunk of size 1916, which is longer than the specified 1500
Created a chunk of size 1642, which is longer than the specified 1500
Created a chunk of size 2868, which is longer than the specified 1500
Created a chunk of size 1639, which is longer than the specified 1500
Created a chunk of size 1594, which is longer than the specified 1500
Created a chunk of s

In [32]:
chunker02 = "Recursive Structure Aware Splitting"

text_splitter = RecursiveCharacterTextSplitter(separators = ["\n\n", "\n"], chunk_size=chunkSize, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

persist_directory = 'chromaDB' + str(chromaCounter) + '/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings(),
    persist_directory=persist_directory
)
vectordb.persist()
docs02 = vectordb.similarity_search_with_score(query, k=2)

In [33]:
chunker03 = "NLP Chunking: Tracking Topic Changes"

text_splitter = NLTKTextSplitter(chunk_size=chunkSize)#, separator="\n")
splits = text_splitter.split_documents(docs)

persist_directory = 'chromaDB' + str(chromaCounter) + '/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings(),
    persist_directory=persist_directory
)
vectordb.persist()
docs03 = vectordb.similarity_search_with_score(query, k=2)

In [34]:
print("question = \"" + query + "\"")

with open("output.txt", "w") as file:
    count = 1
    for result in docs01:
        file.write(f"response{count} = \"{result[0].page_content}\"\n")
        count += 1

    count = 1
    for result in docs02:
        file.write(f"response{count} = \"{result[0].page_content}\"\n")
        count += 1

    count = 1
    for result in docs03:
        file.write(f"response{count} = \"{result[0].page_content}\"\n")
        count += 1

question = "Can I make product bundles on ?"


In [35]:
import re
import textwrap

def clean_text(text):
    # Remove any image file references
    text = re.sub(r"\S+\.(png|jpg|jpeg|gif)\s*", "", text)

    # Normalize spacing issues
    text = re.sub(r"\s+", " ", text).strip()

    # Correct common typographical errors
    text = re.sub(r"isnot", "is not", text)
    text = re.sub(r"orBrand", "or Brand", text)
    text = re.sub(r"ourWarranty", "our Warranty", text)

    # Remove redundant response indicators
    text = re.sub(r"response\d+\s*=\s*\"", "", text)

    # Remove numbers followed by a dot, e.g., "1."
    text = re.sub(r"\d+\.", "", text)

    # Remove numbers followed directly by a dash, e.g., "2-"
    text = re.sub(r"\d+-", "", text)

    # Deduplicate text
    lines = text.split('.')
    seen = set()
    unique_lines = []
    for line in lines:
        line_clean = line.strip()
        if line_clean not in seen:
            seen.add(line_clean)
            unique_lines.append(line_clean)

    # Reconstruct text with clean lines
    cleaned_text = '. '.join(unique_lines).strip()
    if not cleaned_text.endswith('.'):
        cleaned_text += '.'

    # Split into multiple lines with a reasonable width
    wrapped_text = textwrap.fill(cleaned_text, width=100)

    return wrapped_text

with open('output.txt', 'r') as file:
    raw_text = file.read()
    
cleaned_text = clean_text(raw_text)

with open('outputCleaned.txt', 'w') as file:
    file.write(cleaned_text)

In [36]:
loader = TextLoader("outputCleaned.txt")
docs = loader.load()

text_splitter = CharacterTextSplitter(separator="\n", chunk_size=200, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

persist_directory = 'chromaDB' + str(chromaCounter) + '/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings(),
    persist_directory=persist_directory
)
vectordb.persist()
docs = vectordb.similarity_search_with_score(query, k=2)

In [37]:
print("question = \"" + query + "\"")
count = 1
for result in docs:
    print(f"response" + str(count) + " = \"" + result[0].page_content + "\"")  
    count+=1

question = "Can I make product bundles on ?"
response1 = "Bundling? Note: Only one type of bundle feature can be activated at a single time for a particular
SKU. Minimum 5 customs products should be there if a seller is selling only custom made products"
response2 = "What are the types of Product Bundling? Note: Only one type of bundle feature can be activated at a
single time for a particular SKU. You must deactivate one bundle feature for that SKU in order to"


In [38]:
if os.path.exists("/Users/moiz/Moiz/Github/ITA-Project/DarazDataMain.txt"):
    os.remove("/Users/moiz/Moiz/Github/ITA-Project/DarazDataMain.txt")

if os.path.exists("/Users/moiz/Moiz/Github/ITA-Project/DarazDataSeller.txt"):
    os.remove("/Users/moiz/Moiz/Github/ITA-Project/DarazDataSeller.txt")

if os.path.exists("/Users/moiz/Moiz/Github/ITA-Project/output.txt"):
    os.remove("/Users/moiz/Moiz/Github/ITA-Project/output.txt")

In [39]:
# with open('output.txt', 'r') as file:
#     content = file.readlines()

### See Img1 in Images Folder